# Single-Vector 評価

通常のシングルベクトル検索でJQaRAデータセットを評価します。

- **データセット**: JQaRA (Japanese Question-Answer Retrieval Assessment)
- **エンベディング**: sentence-transformers (切り替え可能)
- **評価方式**: 各クエリに対して100件の候補文書をリランキング

## 1. モデル選択とWeaviate接続

使用するエンベディングモデルを選択します。

In [1]:
import weaviate

# ---------- モデル選択 ----------
SUPPORTED_MODELS = {
    "multilingual-e5-small": "intfloat/multilingual-e5-small",      # 384次元
    "multilingual-e5-base": "intfloat/multilingual-e5-base",        # 768次元
    "multilingual-e5-large": "intfloat/multilingual-e5-large",      # 1024次元
    "ruri-v3-310": "cl-nagoya/ruri-v3-310m",
}

# ここでモデルを切り替え
# SELECTED_MODEL = "multilingual-e5-small"
SELECTED_MODEL = "ruri-v3-310"

MODEL_NAME = SUPPORTED_MODELS[SELECTED_MODEL]
print(f"Using model: {MODEL_NAME}")

# Weaviate接続
client = weaviate.connect_to_local()

# コレクション名（モデルごとに分ける）
COLLECTION_NAME = f"jqara_single_{SELECTED_MODEL.replace('-', '_').replace('.', '_')}"
print(f"Collection name: {COLLECTION_NAME}")

Using model: cl-nagoya/ruri-v3-310m
Collection name: jqara_single_ruri_v3_310


## 2. エンベディングモデル読み込み

sentence-transformersでエンベディングモデルを読み込みます。

In [3]:
from sentence_transformers import SentenceTransformer

print(f"Loading model: {MODEL_NAME}")
model = SentenceTransformer(MODEL_NAME)
print(f"Embedding dimension: {model.get_sentence_embedding_dimension()}")

Loading model: cl-nagoya/ruri-v3-310m


ValueError: The checkpoint you are trying to load has model type `modernbert` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

## 3. コレクション作成

シングルベクトル用のコレクションを作成します。
- `q_id`: クエリIDでフィルタリングするためのプロパティ（インデックス有効）
- `vector`: ドキュメントのエンベディングベクトル

In [3]:
from weaviate.classes.config import Configure, Property, DataType, VectorDistances

if client.collections.exists(COLLECTION_NAME):
    client.collections.delete(COLLECTION_NAME)

client.collections.create(
    name=COLLECTION_NAME,
    vectorizer_config=Configure.Vectorizer.none(),
    vector_index_config=Configure.VectorIndex.hnsw(
        distance_metric=VectorDistances.COSINE
    ),
    properties=[
        Property(name="d_id", data_type=DataType.TEXT),
        Property(name="q_id", data_type=DataType.TEXT, index_filterable=True),
        Property(name="title", data_type=DataType.TEXT),
        Property(name="text", data_type=DataType.TEXT),
    ],
)

collection = client.collections.get(COLLECTION_NAME)
print(f"Collection created: {COLLECTION_NAME}")

/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/weaviate/warnings.py:196: DeprecationWarning: Dep024: You are using the `vectorizer_config` argument in `collection.config.create()`, which is deprecated.
            Use the `vector_config` argument instead.
            
  warnings.warn(
/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/weaviate/warnings.py:206: DeprecationWarning: Dep025: You are using the `vector_index_config` argument in `collection.config.create()`, which is deprecated.
            Use the `vector_config` argument instead defining `vector_index_config` as a sub-argument.
            
  warnings.warn(


Collection created: jqara_single_multilingual_e5_small


## 4. JQaRAデータセット読み込み

JQaRAテストデータを読み込みます。各レコードには以下が含まれます：
- `id`: ドキュメントID
- `q_id`: クエリID（同一クエリに対する複数の候補文書をグループ化）
- `label`: 正解ラベル（1=関連, 0=非関連）
- `text`, `title`: 文書内容

In [4]:
import json
import os
from pathlib import Path

dataset = []
# notebookディレクトリからの相対パス
project_root = Path(os.getcwd()).parent
file_path = project_root / "data" / "jqara_test.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)        
        dataset.append(obj)

print(f"Loaded {len(dataset)} records")
print(dataset[0])

Loaded 166700 records
{'id': 'QA20CAPR-1004#5191928', 'q_id': 'QA20CAPR-1004', 'passage_row_id': '5191928', 'label': 1, 'text': '絶対零度(ぜったいれいど、アブソリュートゼロ、英: Absolute zero)は、熱力学上の最低温度である摂氏−273.15度。', 'title': '絶対零度 (曖昧さ回避)', 'question': '摂氏ではマイナス273.15度にあたる、全ての原子の振動が停止する最も低い温度を何というでしょう?', 'answers': ['絶対零度']}


## 5. ドキュメントエンベディング生成とアップロード

ドキュメント(title + text)をエンベディングしてWeaviateにアップロードします。

E5モデルでは、ドキュメントには `passage: ` プレフィックスを付けます。

In [5]:
from tqdm import tqdm

N = len(dataset)  # 全件処理
subset = dataset[:N]

# ドキュメントテキストを準備（E5形式: passage: prefix）
def prepare_doc_text(title: str, text: str) -> str:
    title = (title or "").strip()
    text = (text or "").strip()
    if title and text:
        content = f"{title}\n\n{text}"
    else:
        content = title or text or ""
    return f"passage: {content}"

doc_texts = [prepare_doc_text(obj["title"], obj["text"]) for obj in subset]

print(f"Encoding {len(doc_texts)} documents...")
doc_embeddings = model.encode(doc_texts, show_progress_bar=True, batch_size=64)
print(f"Embeddings shape: {doc_embeddings.shape}")

Encoding 166700 documents...


Batches: 100%|██████████| 2605/2605 [09:09<00:00,  4.74it/s]


Embeddings shape: (166700, 384)


In [6]:
# Weaviateにアップロード
total = len(subset)

with collection.batch.fixed_size(batch_size=100) as batch, tqdm(total=total, desc="📤 Uploading to Weaviate") as pbar:
    for obj, emb in zip(subset, doc_embeddings):
        batch.add_object(
            properties={
                "d_id": obj["id"],
                "q_id": obj["q_id"],
                "text": obj["text"],
                "title": obj["title"],
            },
            vector=emb.tolist(),
        )
        pbar.update(1)

if collection.batch.failed_objects:
    print(f"\n❌ Number of failed imports: {len(collection.batch.failed_objects)}")
    print(f"First failed object: {collection.batch.failed_objects[0]}")
else:
    print("\n✅ All objects uploaded successfully!")

count = collection.aggregate.over_all(total_count=True).total_count
print(f"🧮 現在の登録件数: {count}")

📤 Uploading to Weaviate: 100%|██████████| 166700/166700 [00:50<00:00, 3313.48it/s]



✅ All objects uploaded successfully!
🧮 現在の登録件数: 166700


## 6. 評価用データ構造作成

ranx評価用のqrels（正解ラベル）とqid_to_query（クエリマッピング）を作成します。

In [7]:
from typing import Dict, List, Tuple

qid_to_query: Dict[str, str] = {}
qrels: Dict[str, Dict[str, int]] = {}

cnt = 0
for rec in subset:
    cnt += 1
    qid   = rec["q_id"]
    q = rec.get("question")
    docid = rec.get("id")
    rel   = int(rec.get("label", 0))

    if qid not in qid_to_query:
        qid_to_query[qid] = q
    qrels.setdefault(qid, {})[docid] = rel

print(f"Loaded {cnt} records")
print(f"Unique queries: {len(qid_to_query)}")

Loaded 166700 records
Unique queries: 1667


## 7. 非同期検索ヘルパー関数

全クエリを並列実行してrun_dict（検索結果）とレイテンシを収集する関数を定義します。

In [8]:
from typing import Callable, Awaitable
import asyncio

async def build_run_dict_async_with_qid(
        search_fn: Callable[[str, str, int], Awaitable[Tuple[List[Tuple[str, float]], float]]],
        qid_to_query: Dict[str, str],
        topk: int = 100
    ) -> Tuple[Dict[str, Dict[str, float]], List[float]]:
    """
    非同期検索関数を使って全クエリを実行し、run_dictとlatenciesを返す。
    q_idでフィルタリングするバージョン。
    
    Args:
        search_fn: (q, q_id, topk) -> ([(doc_id, score), ...], elapsed)
        qid_to_query: {qid: query_text} の辞書
        topk: 各クエリで取得する件数
        
    Returns:
        run_dict: {qid: {doc_id: score, ...}, ...}
        latencies: 各クエリのレイテンシリスト
    """
    CONCURRENCY = 10
    sem = asyncio.Semaphore(CONCURRENCY)    
    run_dict: Dict[str, Dict[str, float]] = {}
    latencies: List[float] = []

    async def _one(qid: str, q: str):
        async with sem:
            ranked, elapsed = await search_fn(q, qid, topk)
            return qid, ranked, elapsed

    tasks = [_one(qid, q) for qid, q in qid_to_query.items()]
    
    for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Searching"):
        qid, ranked, elapsed = await coro
        run_dict[qid] = {doc_id: score for doc_id, score in ranked}
        latencies.append(elapsed)
    
    return run_dict, latencies

## 8. シングルベクトル検索関数

q_idでフィルタリングしてシングルベクトル検索を実行する関数を定義します。
- クエリをエンベディングモデルでエンコード（E5形式: `query: ` prefix）
- 該当q_idの候補文書群内でnear_vector検索
- distanceを反転してスコア化（ranxは大きいスコアを上位とするため）

In [9]:
from weaviate.classes.query import MetadataQuery, Filter
import time

async def async_single_vector_search(async_col, q: str, q_id: str, topk: int = 100) -> Tuple[List[Tuple[str, float]], float]:
    """
    シングルベクトルを用いた非同期検索関数
    q_idでフィルタリングして、該当クエリの候補文書群の中からランキング
    
    Args:
        async_col: 非同期Weaviateコレクション
        q: 検索クエリ文字列
        q_id: フィルタ用のクエリID
        topk: 取得する件数
        
    Returns:
        ([(doc_id, score), ...], elapsed) のタプル
        scoreは大きいほど良い（distanceを反転）
    """
    # クエリのベクトル化（E5形式: query: prefix）
    q_text = f"query: {q}"
    q_vec = model.encode(q_text).tolist()

    # 非同期クエリの実行（q_idでフィルタリング）
    start = time.perf_counter()

    response = await async_col.query.near_vector(
        near_vector=q_vec,
        limit=topk,
        filters=Filter.by_property("q_id").equal(q_id),
        return_metadata=MetadataQuery(distance=True)
    )

    elapsed = time.perf_counter() - start

    # 結果の整形: (doc_id, score) のリスト
    # ranxは大きいスコアを上位とするため、distanceを反転（負にする）
    results: List[Tuple[str, float]] = []
    for obj in response.objects:
        doc_id = obj.properties.get("d_id", "")
        score = -obj.metadata.distance  # 距離を反転！
        results.append((doc_id, score))

    return results, elapsed


def create_search_fn(async_col):
    """async_colをバインドした検索関数を返すファクトリ（q_id対応版）"""
    async def search_fn(q: str, q_id: str, topk: int = 100) -> Tuple[List[Tuple[str, float]], float]:
        return await async_single_vector_search(async_col, q, q_id, topk)
    return search_fn

## 9. 全クエリで検索実行

全クエリに対してシングルベクトル検索を実行し、結果とレイテンシを収集します。

In [10]:
from tqdm import tqdm

search_run_dict = {}
search_latencies = []

async with weaviate.use_async_with_local() as async_client:
    async_col = async_client.collections.get(COLLECTION_NAME)
    
    # async_col をバインドした検索関数を作成
    search_fn = create_search_fn(async_col)
    
    # 全クエリを実行（q_idでフィルタリング）
    search_run_dict, search_latencies = await build_run_dict_async_with_qid(
        search_fn, 
        qid_to_query,
        topk=100
    )

print(f"検索完了: {len(search_run_dict)} クエリ")
print(f"平均レイテンシ: {sum(search_latencies) / len(search_latencies):.4f} 秒")

Searching: 100%|██████████| 1667/1667 [00:16<00:00, 102.56it/s]

検索完了: 1667 クエリ
平均レイテンシ: 0.0697 秒


## 10. 評価関数定義

ranxを使用してNDCG, MAP, MRR, Precision, Recallを計算する関数を定義します。

In [11]:
from ranx import Qrels, Run, evaluate

def evaluate_run_dict(run_dict: Dict[str, Dict[str, float]], qrels: Dict[str, Dict[str, int]]):
    qrels_ranx = Qrels(qrels)
    run_ranx   = Run(run_dict)

    metrics = [
        "ndcg@1", "ndcg@3", "ndcg@5", "ndcg@10",
        "map", "mrr", "mrr@10", "precision@10", "recall@10", "recall@100",
    ]

    scores = evaluate(qrels_ranx, run_ranx, metrics=metrics)
    for m in metrics:
        print(f"{m:>12}: {scores[m]:.4f}")

## 11. レイテンシ統計関数

In [12]:
import numpy as np

def print_latency_stats(latencies: List[float]):
    l = np.array(latencies)

    print(f"Count    : {len(l)}")
    print(f"Mean     : {l.mean():.4f} sec")
    print(f"Median   : {np.median(l):.4f} sec")
    print(f"p90      : {np.percentile(l,90):.4f} sec")
    print(f"p95      : {np.percentile(l,95):.4f} sec")
    print(f"p99      : {np.percentile(l,99):.4f} sec")
    print(f"Min/Max  : {l.min():.4f} / {l.max():.4f} sec")

## 12. 評価結果

シングルベクトル検索の精度とレイテンシを表示します。

In [13]:
print(f"=== {SELECTED_MODEL} ===")
print(f"Model: {MODEL_NAME}")
print()
evaluate_run_dict(search_run_dict, qrels)
print()
print_latency_stats(search_latencies)

=== multilingual-e5-small ===
Model: intfloat/multilingual-e5-small

      ndcg@1: 0.6293
      ndcg@3: 0.5417
      ndcg@5: 0.5090
     ndcg@10: 0.4914
         map: 0.4284
         mrr: 0.7317
      mrr@10: 0.7283
precision@10: 0.3312
   recall@10: 0.4133
  recall@100: 1.0000

Count    : 1667
Mean     : 0.0697 sec
Median   : 0.0633 sec
p90      : 0.0941 sec
p95      : 0.1181 sec
p99      : 0.2095 sec
Min/Max  : 0.0124 / 0.7344 sec
